# Feature Selection and Cross validation

In [1]:
import numpy as np
import pandas as pd

In [2]:
import itertools
from scipy.special import comb

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [4]:
from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [5]:
credit = pd.read_csv('Credit.csv')
credit[:5]

,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity,Balance
0,14.891,3606,283,2,34,11,Male,No,Yes,Caucasian,333
1,106.025,6645,483,3,82,15,Female,Yes,Yes,Asian,903
2,104.593,7075,514,4,71,11,Male,No,No,Asian,580
3,148.924,9504,681,3,36,11,Female,No,No,Asian,964
4,55.882,4897,357,2,68,16,Male,No,Yes,Caucasian,331


In [6]:
credit.shape

(400, 11)

In [7]:
credit['Ethnicity'].value_counts()

Caucasian           199
Asian               102
African American     99
Name: Ethnicity, dtype: int64

### Encode categorical vars

In [8]:
credit[:5]

,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity,Balance
0,14.891,3606,283,2,34,11,Male,No,Yes,Caucasian,333
1,106.025,6645,483,3,82,15,Female,Yes,Yes,Asian,903
2,104.593,7075,514,4,71,11,Male,No,No,Asian,580
3,148.924,9504,681,3,36,11,Female,No,No,Asian,964
4,55.882,4897,357,2,68,16,Male,No,Yes,Caucasian,331


In [9]:
credit1 = pd.get_dummies(credit,
                         columns = ['Gender','Student','Married','Ethnicity'],
                         drop_first = True)
credit1[:5]

,Income,Limit,Rating,Cards,Age,Education,Balance,Gender_Female,Student_Yes,Married_Yes,Ethnicity_Asian,Ethnicity_Caucasian
0,14.891,3606,283,2,34,11,333,0,0,1,0,1
1,106.025,6645,483,3,82,15,903,1,1,1,1,0
2,104.593,7075,514,4,71,11,580,0,0,0,1,0
3,148.924,9504,681,3,36,11,964,1,0,0,1,0
4,55.882,4897,357,2,68,16,331,0,0,1,0,1


In [10]:
credit1.shape

(400, 12)

In [11]:
credit2 = credit1.drop('Balance',axis=1)
credit2[:5]

,Income,Limit,Rating,Cards,Age,Education,Gender_Female,Student_Yes,Married_Yes,Ethnicity_Asian,Ethnicity_Caucasian
0,14.891,3606,283,2,34,11,0,0,1,0,1
1,106.025,6645,483,3,82,15,1,1,1,1,0
2,104.593,7075,514,4,71,11,0,0,0,1,0
3,148.924,9504,681,3,36,11,1,0,0,1,0
4,55.882,4897,357,2,68,16,0,0,1,0,1


In [12]:
credit2.shape

(400, 11)

In [13]:
# How many models with 2 predictors?
comb(11,2)

55.0

In [14]:
# How many models with 3 predictors?
comb(11,3)

165.0

In [15]:
# How many models overall?
2**11

2048

## Split into train and test sets

In [16]:
y0 = credit1.Balance
X0 = credit1.drop(columns = 'Balance', axis = 1)

In [17]:
X,X_test,y,y_test = train_test_split(X0,y0,
                                     test_size = 0.25,
                                     random_state=1)

In [18]:
# train sets are X,y
# test sets are X_test,y_test

In [19]:
print(X.shape,X_test.shape)

(300, 11) (100, 11)


In [20]:
# train set size
n = 300

## Build Model with all predictors

### Holdout Cross Validation

In [21]:
model0 = LinearRegression().fit(X,y)

In [22]:
# MSPE

In [23]:
yhat0 = model0.predict(X_test)
MSPE = mean_squared_error(y_test,yhat0)
MSPE

11521.699081990417

In [24]:
np.sqrt(MSPE)

107.33917775905691

### K-fold Cross Validation (K = 10)

In [25]:
# Split all rows of data into 10 folds

In [26]:
kfold = KFold(n_splits=10,random_state=1,shuffle = True)

In [27]:
mspe = cross_val_score(LinearRegression(),X0,y0,
                       cv = kfold,
                       scoring = 'neg_mean_squared_error')

In [28]:
mspe

array([ -9977.31246066, -16049.63211923,  -8256.55274834, -10718.06396806,
        -9983.7567259 , -10455.00758441,  -9675.20399605,  -9789.97445626,
        -6454.62129103, -10230.20348514])

In [29]:
-mspe.mean()

10159.032883508191

In [30]:
np.sqrt(-mspe.mean())

100.79202787675318

## Find best models based on AIC and Adj R-squared

In [31]:
# A function returning SSE, R-squared

In [32]:
def get_sse(X,Y):
    model = LinearRegression().fit(X,Y)
    yhat = model.predict(X)
    SSE = mean_squared_error(Y,yhat)*n
    R_squared = model.score(X,Y)
    return SSE, R_squared

In [33]:
# Get SSE, R-squared for the full model
SSE, R_squared = get_sse(X,y)

In [34]:
SSE

2695832.2458131495

In [35]:
R_squared

0.9557279779952305

In [36]:
list(X.columns)

['Income',
 'Limit',
 'Rating',
 'Cards',
 'Age',
 'Education',
 'Gender_Female',
 'Student_Yes',
 'Married_Yes',
 'Ethnicity_Asian',
 'Ethnicity_Caucasian']

In [37]:
# all sets of two column names

In [38]:
list(itertools.combinations(X.columns,2))

[('Income', 'Limit'),
 ('Income', 'Rating'),
 ('Income', 'Cards'),
 ('Income', 'Age'),
 ('Income', 'Education'),
 ('Income', 'Gender_Female'),
 ('Income', 'Student_Yes'),
 ('Income', 'Married_Yes'),
 ('Income', 'Ethnicity_Asian'),
 ('Income', 'Ethnicity_Caucasian'),
 ('Limit', 'Rating'),
 ('Limit', 'Cards'),
 ('Limit', 'Age'),
 ('Limit', 'Education'),
 ('Limit', 'Gender_Female'),
 ('Limit', 'Student_Yes'),
 ('Limit', 'Married_Yes'),
 ('Limit', 'Ethnicity_Asian'),
 ('Limit', 'Ethnicity_Caucasian'),
 ('Rating', 'Cards'),
 ('Rating', 'Age'),
 ('Rating', 'Education'),
 ('Rating', 'Gender_Female'),
 ('Rating', 'Student_Yes'),
 ('Rating', 'Married_Yes'),
 ('Rating', 'Ethnicity_Asian'),
 ('Rating', 'Ethnicity_Caucasian'),
 ('Cards', 'Age'),
 ('Cards', 'Education'),
 ('Cards', 'Gender_Female'),
 ('Cards', 'Student_Yes'),
 ('Cards', 'Married_Yes'),
 ('Cards', 'Ethnicity_Asian'),
 ('Cards', 'Ethnicity_Caucasian'),
 ('Age', 'Education'),
 ('Age', 'Gender_Female'),
 ('Age', 'Student_Yes'),
 ('Age'

In [39]:
p = len(X.columns)
p

11

In [40]:
list(range(1,p+1))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [41]:
# Create 2**11 regression models
# getting their R-squared and SSE

In [42]:
SSE_list, R2_list, feature_list, num_features = [],[],[],[]

In [43]:
for k in range(1,p+1):
    for subset in itertools.combinations(X.columns,k):
        num_features.append(k)
        feature_list.append(subset)
        
        X2 = X[list(subset)]
        SSE, R_squared = get_sse(X2,y)
        
        SSE_list.append(SSE)
        R2_list.append(R_squared)

In [44]:
len(num_features)

2047

In [45]:
num_features[:5]

[1, 1, 1, 1, 1]

In [46]:
feature_list[:5]

[('Income',), ('Limit',), ('Rating',), ('Cards',), ('Age',)]

In [47]:
R2_list[:5]

[0.2200181327359455,
 0.7568493163363378,
 0.7616305879459113,
 0.014850886580138,
 0.0005190442873671541]

In [48]:
SSE_list[:5]

[47495013.18673583,
 14806042.82183892,
 14514899.44059262,
 59988151.13087408,
 60860852.23746908]

In [49]:
# create dataframe with the four lists

In [50]:
zip1 = zip(num_features,feature_list,R2_list,SSE_list)
df = pd.DataFrame(list(zip1),
                  columns = ['n_features','features','R-squared','SSE'])

In [51]:
df['SSE'] = df['SSE'].round(0)
df[:5]

,n_features,features,R-squared,SSE
0,1,"(Income,)",0.220018,47495013.0
1,1,"(Limit,)",0.756849,14806043.0
2,1,"(Rating,)",0.761631,14514899.0
3,1,"(Cards,)",0.014851,59988151.0
4,1,"(Age,)",0.000519,60860852.0


In [52]:
df.shape

(2047, 4)

In [53]:
2**11 - 1

2047

We just built 2047 regression models. Lets see few of them.

In [54]:
df.sample(9, random_state=1)

,n_features,features,R-squared,SSE
309,4,"(Income, Cards, Gender_Female, Ethnicity_Cauca...",0.242865,46103833.0
1199,6,"(Income, Rating, Age, Gender_Female, Ethnicity...",0.879609,7330923.0
1755,7,"(Limit, Cards, Age, Education, Gender_Female, ...",0.771386,13920841.0
563,5,"(Income, Limit, Rating, Cards, Gender_Female)",0.878893,7374524.0
56,2,"(Gender_Female, Student_Yes)",0.073873,56394170.0
546,4,"(Education, Gender_Female, Student_Yes, Marrie...",0.075743,56280289.0
1293,6,"(Limit, Rating, Cards, Education, Gender_Femal...",0.765966,14250915.0
181,3,"(Cards, Education, Gender_Female)",0.025366,59347877.0
613,5,"(Income, Limit, Age, Education, Ethnicity_Asian)",0.872174,7783615.0


In [55]:
pd.set_option('display.max_colwidth',190)

In [56]:
df.sample(9, random_state=1)

,n_features,features,R-squared,SSE
309,4,"(Income, Cards, Gender_Female, Ethnicity_Caucasian)",0.242865,46103833.0
1199,6,"(Income, Rating, Age, Gender_Female, Ethnicity_Asian, Ethnicity_Caucasian)",0.879609,7330923.0
1755,7,"(Limit, Cards, Age, Education, Gender_Female, Married_Yes, Ethnicity_Caucasian)",0.771386,13920841.0
563,5,"(Income, Limit, Rating, Cards, Gender_Female)",0.878893,7374524.0
56,2,"(Gender_Female, Student_Yes)",0.073873,56394170.0
546,4,"(Education, Gender_Female, Student_Yes, Married_Yes)",0.075743,56280289.0
1293,6,"(Limit, Rating, Cards, Education, Gender_Female, Ethnicity_Caucasian)",0.765966,14250915.0
181,3,"(Cards, Education, Gender_Female)",0.025366,59347877.0
613,5,"(Income, Limit, Age, Education, Ethnicity_Asian)",0.872174,7783615.0


In [57]:
df[-6:]

,n_features,features,R-squared,SSE
2041,10,"(Income, Limit, Rating, Cards, Education, Gender_Female, Student_Yes, Married_Yes, Ethnicity_Asian, Ethnicity_Caucasian)",0.954878,2747582.0
2042,10,"(Income, Limit, Rating, Age, Education, Gender_Female, Student_Yes, Married_Yes, Ethnicity_Asian, Ethnicity_Caucasian)",0.954477,2772027.0
2043,10,"(Income, Limit, Cards, Age, Education, Gender_Female, Student_Yes, Married_Yes, Ethnicity_Asian, Ethnicity_Caucasian)",0.954157,2791499.0
2044,10,"(Income, Rating, Cards, Age, Education, Gender_Female, Student_Yes, Married_Yes, Ethnicity_Asian, Ethnicity_Caucasian)",0.953555,2828153.0
2045,10,"(Limit, Rating, Cards, Age, Education, Gender_Female, Student_Yes, Married_Yes, Ethnicity_Asian, Ethnicity_Caucasian)",0.830560,10317590.0
2046,11,"(Income, Limit, Rating, Cards, Age, Education, Gender_Female, Student_Yes, Married_Yes, Ethnicity_Asian, Ethnicity_Caucasian)",0.955728,2695832.0


In [58]:
# Largest R-squared by n_features

In [59]:
best_r2 = df.groupby(['n_features'])['R-squared'].max()
best_r2

n_features
1     0.761631
2     0.877077
3     0.951700
4     0.952881
5     0.954163
6     0.955137
7     0.955429
8     0.955688
9     0.955719
10    0.955724
11    0.955728
Name: R-squared, dtype: float64

In [60]:
# Best model with one feature
df2 = df[df['R-squared'] == best_r2[1]]
df2

,n_features,features,R-squared,SSE
2,1,"(Rating,)",0.761631,14514899.0


In [61]:
# Best model with two features
df2 = df[df['R-squared'] == best_r2[2]]
df2

,n_features,features,R-squared,SSE
12,2,"(Income, Rating)",0.877077,7485072.0


In [62]:
for i in range(1,12):
    df2 = df[df['R-squared'] == best_r2[i]]
    print(df2.index.values)

[2]
[12]
[79]
[235]
[564]
[1025]
[1495]
[1833]
[1988]
[2039]
[2046]


In [63]:
list1 = list()
for i in range(1,12):
    df2 = df[df['R-squared'] == best_r2[i]]
    list1.extend(df2.index.values.tolist())

In [64]:
list1

[2, 12, 79, 235, 564, 1025, 1495, 1833, 1988, 2039, 2046]

In [65]:
df2 = df.iloc[list1].copy()
df2

,n_features,features,R-squared,SSE
2,1,"(Rating,)",0.761631,14514899.0
12,2,"(Income, Rating)",0.877077,7485072.0
79,3,"(Income, Rating, Student_Yes)",0.951700,2941135.0
235,4,"(Income, Limit, Rating, Student_Yes)",0.952881,2869207.0
564,5,"(Income, Limit, Rating, Cards, Student_Yes)",0.954163,2791149.0
1025,6,"(Income, Limit, Rating, Cards, Age, Student_Yes)",0.955137,2731792.0
1495,7,"(Income, Limit, Rating, Cards, Age, Student_Yes, Ethnicity_Asian)",0.955429,2714033.0
1833,8,"(Income, Limit, Rating, Cards, Age, Student_Yes, Ethnicity_Asian, Ethnicity_Caucasian)",0.955688,2698241.0
1988,9,"(Income, Limit, Rating, Cards, Age, Education, Student_Yes, Ethnicity_Asian, Ethnicity_Caucasian)",0.955719,2696376.0
2039,10,"(Income, Limit, Rating, Cards, Age, Education, Student_Yes, Married_Yes, Ethnicity_Asian, Ethnicity_Caucasian)",0.955724,2696091.0


In [66]:
# best predictor is Rating,   followed by Income. 
# Worst predictor is Gender.

In [67]:
# Add new columns for AIC, adj R-squared 

In [68]:
df2['adj_R-squared'] = 1 - ((1-df2['R-squared'])*(n-1)/
                            (n - df2['n_features'] - 1))
df2['AIC'] = n * np.log(df2['SSE']/n) + 2*df2['n_features']

In [69]:
# Remove R-squared, SSE columns

In [70]:
df2.drop(['R-squared','SSE'],axis=1,inplace=True)

In [71]:
df2

,n_features,features,adj_R-squared,AIC
2,1,"(Rating,)",0.760831,3238.071117
12,2,"(Income, Rating)",0.876249,3041.391616
79,3,"(Income, Rating, Student_Yes)",0.951210,2763.157093
235,4,"(Income, Limit, Rating, Student_Yes)",0.952242,2757.729130
564,5,"(Income, Limit, Rating, Cards, Student_Yes)",0.953383,2751.454427
1025,6,"(Income, Limit, Rating, Cards, Age, Student_Yes)",0.954219,2747.005766
1495,7,"(Income, Limit, Rating, Cards, Age, Student_Yes, Ethnicity_Asian)",0.954361,2747.049141
1833,8,"(Income, Limit, Rating, Cards, Age, Student_Yes, Ethnicity_Asian, Ethnicity_Caucasian)",0.954470,2747.298449
1988,9,"(Income, Limit, Rating, Cards, Age, Education, Student_Yes, Ethnicity_Asian, Ethnicity_Caucasian)",0.954345,2749.091020
2039,10,"(Income, Limit, Rating, Cards, Age, Education, Student_Yes, Married_Yes, Ethnicity_Asian, Ethnicity_Caucasian)",0.954192,2751.059309


### AIC best model

In [72]:
row6 = df2[df2.AIC == df2.AIC.min()]
row6

,n_features,features,adj_R-squared,AIC
1025,6,"(Income, Limit, Rating, Cards, Age, Student_Yes)",0.954219,2747.005766


In [73]:
# get predictor names of AIC best model

In [74]:
row6.features.iloc[0]

('Income', 'Limit', 'Rating', 'Cards', 'Age', 'Student_Yes')

In [75]:
list1 = list(row6.features.iloc[0])
list1

['Income', 'Limit', 'Rating', 'Cards', 'Age', 'Student_Yes']

In [76]:
# make dataset with these columns only

In [77]:
X1 = X[list1]
X1[:5]

,Income,Limit,Rating,Cards,Age,Student_Yes
82,23.672,4433,344,3,63,0
367,23.793,3615,263,2,70,0
179,58.026,7499,560,5,67,0
27,32.793,4534,333,2,44,0
89,59.530,7518,543,3,52,0


### Adj-R2 best model

In [78]:
max_R2 = df2['adj_R-squared'].max()
max_R2

0.9544702381591676

In [79]:
df2[df2['adj_R-squared'] == max_R2]

,n_features,features,adj_R-squared,AIC
1833,8,"(Income, Limit, Rating, Cards, Age, Student_Yes, Ethnicity_Asian, Ethnicity_Caucasian)",0.95447,2747.298449


In [80]:
# Adj-R2 best model has 8 features

In [81]:
row8 = df2[df2['adj_R-squared'] == max_R2]
row8

,n_features,features,adj_R-squared,AIC
1833,8,"(Income, Limit, Rating, Cards, Age, Student_Yes, Ethnicity_Asian, Ethnicity_Caucasian)",0.95447,2747.298449


In [82]:
# get feature names of adj-R2 best model

In [83]:
row8.features.iloc[0]

('Income',
 'Limit',
 'Rating',
 'Cards',
 'Age',
 'Student_Yes',
 'Ethnicity_Asian',
 'Ethnicity_Caucasian')

In [84]:
list2 = list(row8.features.iloc[0])
list2

['Income',
 'Limit',
 'Rating',
 'Cards',
 'Age',
 'Student_Yes',
 'Ethnicity_Asian',
 'Ethnicity_Caucasian']

## Holdout Validation

In [85]:
# SQ(MSPE) of best AIC model

In [86]:
X1 = X[list1]

In [87]:
model1 = LinearRegression().fit(X1,y)

In [88]:
X1_test = X_test[list1]
yhat1 = model1.predict(X1_test) 

In [89]:
MSPE = mean_squared_error(y_test,yhat1)
np.sqrt(MSPE)

105.78913727639262

In [90]:
# SQ(MSPE) of best Adj R-squared model

In [91]:
X2 = X[list2]

In [92]:
model2 = LinearRegression().fit(X2,y)

In [93]:
X2_test = X_test[list2]
yhat2 = model2.predict(X2_test)

In [94]:
MSPE = mean_squared_error(y_test,yhat2)
np.sqrt(MSPE)

107.37550073954556

In [95]:
# AIC model is best predictive model

## K-fold Cross Validation (K = 10)

In [96]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

### full model -dataset X0

In [97]:
kfold = KFold(n_splits=10,random_state=1,shuffle = True)

In [98]:
# Use all rows of data (Data sets X0,y0)

In [99]:
mspe = cross_val_score(LinearRegression(),X0,y0,
                       cv = kfold,
                       scoring = 'neg_mean_squared_error')

In [100]:
-mspe.mean(), 

(10159.032883508191,)

In [101]:
np.sqrt(-mspe.mean())

100.79202787675318

### best AIC model -dataset X1

In [102]:
X1 = X0[list1]

In [103]:
mspe = cross_val_score(LinearRegression(),X1,y0,
                       cv = kfold,
                       scoring = 'neg_mean_squared_error')

In [104]:
-mspe.mean()

10066.384682386239

In [105]:
np.sqrt(-mspe.mean())

100.33137436707543

### best adj-R2 model -dataset X2

In [106]:
X2 = X0[list2]

In [107]:
mspe = cross_val_score(LinearRegression(),X2,y0,
                       cv = kfold,
                       scoring = 'neg_mean_squared_error')

In [108]:
-mspe.mean()

10102.635473660017

In [109]:
np.sqrt(-mspe.mean())

100.5118673274953

In [110]:
# AIC model has smallest mspe, it is best predictive model

## Predict Price of a new house with the best AIC Model

In [111]:
newval = X1.head(1).copy()
newval

,Income,Limit,Rating,Cards,Age,Student_Yes
0,14.891,3606,283,2,34,0


In [112]:
newval.Income = np.median(X1.Income)
newval.Limit = np.median(X1.Limit)
newval.Rating = np.median(X1.Rating)
newval.Cards = np.median(X1.Cards)
newval.Age = np.median(X1.Age)

In [113]:
# most common student status

In [114]:
pd.value_counts(X1.Student_Yes)

0    360
1     40
Name: Student_Yes, dtype: int64

In [115]:
newval.Student_Yes = 0

In [116]:
newval

,Income,Limit,Rating,Cards,Age,Student_Yes
0,33.1155,4622.5,344.0,3.0,56.0,0


In [117]:
# fit model and predict Balance

In [118]:
model = LinearRegression().fit(X1,y0)
model.predict(newval)

array([538.52330854])

Predicted Customer Balance is 538.52 dollars